In [125]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Conv1D, MaxPooling1D
from tensorflow.keras.layers import LSTM, Embedding, GRU
import tensorflow as tf

In [126]:
#Baca file csv(file berasal dari kaggle https://www.kaggle.com/uciml/news-aggregator-dataset)
df = pd.read_csv('/content/sample_data/uci-news-aggregator.csv', usecols=['TITLE', 'CATEGORY'])


In [127]:
#melihat struktur 
df.head()
#df.tail()

,TITLE,CATEGORY
0,"Fed official says weak data caused by weather,...",b
1,Fed's Charles Plosser sees high bar for change...,b
2,US open: Stocks fall after Fed official hints ...,b
3,"Fed risks falling 'behind the curve', Charles ...",b
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,b


In [128]:
#melihat jumlah katagori 
df.CATEGORY.value_counts()

e    21901
b    19391
t    16093
m     7528
Name: CATEGORY, dtype: int64

Karena label kita berupa data kategorikal, maka kita perlu melakukan proses one-hot-encoding

In [129]:
df['ca_labels'] = df['CATEGORY'].map({'b':0, 't':1, 'e':2, 'm':3})

In [130]:
df['ca_labels']

0        0
1        0
2        0
3        0
4        0
        ..
64908    1
64909    1
64910    1
64911    1
64912    1
Name: ca_labels, Length: 64913, dtype: int64

In [131]:
#Agar dapat diproses oleh model, kita perlu mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.
y = df['ca_labels'].values


In [132]:
#melakukan splitting
x_train, x_test, y_train, y_test = train_test_split(df['TITLE'], y,test_size=0.2)

In [134]:
#Kemudian kita ubah setiap kata pada dataset kita ke dalam bilangan numerik dengan fungsi Tokenizer
vocab_size = 2000
embedded_dim = 8
max_len = 120
trunc_type = 'post'
oov_tok = "<OOV>"


In [135]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

In [136]:
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(x_train)
len(word_index)

24602

In [137]:
#Setelah tokenisasi selesai, kita perlu membuat mengonversi setiap sampel menjadi sequence.
padded = pad_sequences(sequences,maxlen=max_len, truncating=trunc_type)
padded.shape

(51930, 120)

In [138]:
testing_sequences = tokenizer.texts_to_sequences(x_test)
testing_padded = pad_sequences(testing_sequences, maxlen=max_len)
testing_padded.shape

(12983, 120)

In [139]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedded_dim,input_length=max_len),
    tf.keras.layers.LSTM(15, return_sequences=True),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

In [143]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.92):
      print("\nAkurasi telah mencapai >92%!")
      self.model.stop_training = True
callbacks = myCallback()

In [144]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam', )


In [145]:
#Bismillah
hist = model.fit(padded,y_train,batch_size=64, epochs=10, validation_data=(testing_padded,y_test), validation_batch_size=64, callbacks=[callbacks])

Epoch 1/10
812/812 [==============================] - 40s 47ms/step - loss: 0.2211 - acc: 0.9207 - val_loss: 0.2434 - val_acc: 0.9141

Akurasi telah mencapai >92%!
